# IV) Constraint imposition

In [1]:
from ngsolve import *
from ngsolve.webgui import Draw

____
# 1) Geometry

In [2]:
from utils.myGeometries import capacitor
mesh = capacitor()
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

____
## 1) Substitution

We want to solve the following Poisson problem with non-zero boundary conditions
$$ 
\left \{\begin{array}{rcll}
-\nabla \cdot (\epsilon_0 \nabla u) & = & 0 & \text{in}~\Omega_0  ~\text{(air)} \\
-\nabla \cdot (3 \epsilon_0 \nabla u) & = & 0 & \text{in}~\Omega_1  ~\text{(dielectric)} \\
u & =  & -1 V& \text{on}~\Gamma_{L} ~\text{(left)} \\
u & =  & +1 V& \text{on}~\Gamma_{R} ~\text{(right)} \\
\vec{D}\cdot \vec{n} & =  &0 & \text{on}~\Gamma_{N} ~\text{(bottom and top)} \\
\end{array} \right.
$$

### a) Function space

In [ ]:
fes = ..............
u, v = fes.TnT()  # Trial aNd Test functions

### b) Integral formulation

In [ ]:
eps0 = 8.85e-12 # F/m
bf = BilinearForm(fes)
bf += ..............
bf.Assemble()

lf = LinearForm(fes) # lf is zero !
lf.Assemble()

## c) Boundary conditions
***How to impose non-zero Dirichlet conditions?***

There are several ways. The easiest one is the following:

- **split** the solution
$$u = u_0 + u_D $$
with $u_0$ the unknown Dofs satisfying zero Dirichlet boundary conditions, and $u_D$ being non-zero known dofs on the boundary
- Then
$$ A(u_0 + u_D) = f $$
- and solve
$$ A u_0 = f - A u_D $$
- So the solution is finally $u = u_0 + u_D$

In [ ]:
u0 = GridFunction(fes)
uD = GridFunction(fes)
uD.Set(.............. , definedon = mesh.Boundaries("left|right")) # set -1 on left, +1 on right
Draw(uD)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [6]:
u0.vec.data = bf.mat.Inverse(inverse = "sparsecholesky", freedofs = fes.FreeDofs()) * (lf.vec - bf.mat * uD.vec)
Draw(u0)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
uSol = ..............
Draw(uSol, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

_______
## 2) Lagrange multiplier

$$ \left\{ \begin{array}{lll}
\int_\Omega \nabla v \cdot \epsilon \nabla u & + \int_{\Gamma_{R}} \lambda v  \;ds + \int_{\Gamma_{L}} \lambda v  \;ds &= 0\\
\int_{\Gamma_{R}} \mu u  \;ds + \int_{\Gamma_{L}} \mu u  \;ds & & =  \int_{\Gamma_{R}} \mu \cdot 1   \;ds -  \int_{\Gamma_{L}} \mu \cdot 1   \;ds  \\
\end{array} \right.
$$

In [8]:
Hprimal = H1(mesh)
Hdual = H1(mesh, definedon=mesh.Boundaries("left|right"))
fes = Hprimal*Hdual

In [ ]:
u, lam = fes.TrialFunction()
v, mu = fes.TestFunction()

bf = BilinearForm(fes)
bf += ............
bf.Assemble()

lf = LinearForm(fes)
lf += .............
lf.Assemble()

In [ ]:
sol = GridFunction(fes)
sol.vec.data = bf.mat.Inverse(freedofs = fes.FreeDofs(), inverse = "sparsecholesky") * lf.vec

- Can the system be solved using `inverse = "sparsecholesky"`? why?

Now with another type of solver (iterative):

In [ ]:
from ngsolve.solvers import GMRes
sol = GridFunction(fes)
u0 = sol.vec.CreateVector(); u0[:] = 0
GMRes(A=bf.mat,  b=lf.vec, x=u0, freedofs = fes.FreeDofs(), tol=1e-15, printrates=True, maxsteps=1000)
sol.vec.data = u0

In [11]:
Draw(sol.components[0], mesh) # primal

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

**Note 1** : the dual variable (Lagrange multiplier) has a physical interpretation (conjugated variable necessary to impose the constraint)

**Note 2** : if need to do a lot of simulations with the same core problem but different constraints, using Schur complement can help to accelerate the computations.

https://mice.cs.columbia.edu/getTechreport.php?techreportID=1581&format=pdf